In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,49463.07,-0.08,0.00,-0.19,0.02,0.01,0.02,-0.01,-0.00
3,-0.08,8580.60,0.45,-0.02,0.04,-0.00,0.00,0.00,0.00
4,0.00,0.45,1785.98,0.01,0.02,-0.00,-0.00,-0.00,0.00
5,-0.19,-0.02,0.01,284.34,0.00,0.00,-0.00,-0.00,-0.00
6,0.02,0.04,0.02,0.00,70.42,0.00,-0.00,0.00,-0.00
7,0.01,-0.00,-0.00,0.00,0.00,23.23,0.00,-0.00,-0.00
8,0.02,0.00,-0.00,-0.00,-0.00,0.00,9.53,-0.00,0.00
9,-0.01,0.00,-0.00,-0.00,0.00,-0.00,-0.00,1.70,-0.00
10,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.18


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00000,0.00000,-0.00005,0.00001,0.00001,0.00002,-0.00005,-0.00004
3,-0.00000,1.00000,0.00011,-0.00001,0.00005,-0.00000,0.00001,0.00001,0.00004
4,0.00000,0.00011,1.00000,0.00001,0.00006,-0.00001,-0.00000,-0.00002,0.00004
5,-0.00005,-0.00001,0.00001,1.00000,0.00003,0.00006,-0.00004,-0.00006,-0.00004
6,0.00001,0.00005,0.00006,0.00003,1.00000,0.00002,-0.00004,0.00000,-0.00004
7,0.00001,-0.00000,-0.00001,0.00006,0.00002,1.00000,0.00002,-0.00001,-0.00003
8,0.00002,0.00001,-0.00000,-0.00004,-0.00004,0.00002,1.00000,-0.00003,0.00001
9,-0.00005,0.00001,-0.00002,-0.00006,0.00000,-0.00001,-0.00003,1.00000,-0.00005
10,-0.00004,0.00004,0.00004,-0.00004,-0.00004,-0.00003,0.00001,-0.00005,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.7015234521219239

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.71981674e+08 8.05653178e+06 3.41704180e+05 8.43999710e+03
 4.98968523e+02 5.13617115e+01 7.46163156e+00 1.80383365e-01
 1.46685919e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999740,-0.022339,-0.004499,-0.000719,-0.000177,-0.000058,-0.000024,-0.000005,-5.407162e-07
1,0.022185,0.999259,-0.031078,-0.004722,-0.001163,-0.000385,-0.000158,-0.000028,-2.833261e-06
2,0.005164,0.030821,0.999113,-0.027347,-0.006574,-0.002170,-0.000890,-0.000159,-1.649403e-05
3,0.000950,0.005459,0.026758,0.998086,-0.052636,-0.016206,-0.006622,-0.001180,-1.263119e-04
4,0.000276,0.001586,0.007641,0.050482,0.993887,-0.091822,-0.033377,-0.005807,-6.211831e-04
5,0.000110,0.000634,0.003047,0.019210,0.084216,0.982728,-0.162040,-0.022628,-2.402196e-03
6,0.000065,0.000374,0.001798,0.011311,0.046186,0.155495,0.981962,-0.095981,-9.755248e-03
7,0.000017,0.000097,0.000470,0.002951,0.011901,0.036168,0.089319,0.989429,-1.076779e-01
8,0.000004,0.000021,0.000101,0.000635,0.002560,0.007758,0.018897,0.106168,9.941349e-01


In [13]:

cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
min_cos_dist = [np.min(row) for row in cos_dist]
arg_min_cos_dist = [np.argmin(row) for row in cos_dist]
min_cos_dist

[0.0002599489646044617,
 0.0007411840036666284,
 0.0008869935275132024,
 0.0019144136770918863,
 0.006113094763977256,
 0.017272001615756816,
 0.01803796994927931,
 0.010571425082966934,
 0.005865131510759514]

In [14]:
arg_min_cos_dist

[0, 1, 2, 3, 4, 5, 6, 7, 8]